# Standardize the format of the global datasets

We have data in several formats (netCDF, csv, gdf) which can be hard to deal with. Most of our global data are grids and could be distributed as netCDF files to make the students' lives easier.

In this notebook, we'll load these datasets, clean them up and combine them (when necessary), and then export them to netCDF.

In [1]:
from pathlib import Path
import numpy as np
import xarray as xr
import harmonica as hm
import boule as bl

In [2]:
data_dir = Path("../data")

## Global gravity and topography

We'll start with the gravity, topography, and geoid data from ICGEM. After loading, copy the `attrs` to the `DataArray` so that it doesn't get lost when merging all of the datasets.

In [3]:
geoid = hm.load_icgem_gdf(data_dir / "EIGEN-6C4-geoid-0.5deg.gdf")
geoid.geoid.attrs = geoid.attrs

In [4]:
gravity = hm.load_icgem_gdf(data_dir / "EIGEN-6C4-gravity-earth-0.5deg.gdf").rename(
    {"h_over_geoid": "height", "gravity_earth": "gravity"}
)
gravity.gravity.attrs = gravity.attrs

In [5]:
topography = hm.load_icgem_gdf(data_dir / "etopo1-0.5deg.gdf").rename(
    {"topography_grd": "topography"}
)
topography.topography.attrs = topography.attrs

In [6]:
topography_smooth = hm.load_icgem_gdf(data_dir / "etopo1-0.5deg-filtered.gdf").rename(
    {"topography_shm": "topography"}
)
topography_smooth.topography.attrs = topography_smooth.attrs

Convert all heights to geometric by subtracting the geoid height.

In [7]:
gravity["height"] += geoid.geoid
topography["topography"] += geoid.geoid
topography_smooth["topography"] += geoid.geoid

Calculate the gravity disturbance.

In [8]:
disturbance = gravity.gravity - bl.WGS84.normal_gravity(gravity.latitude, gravity.height)
disturbance.attrs = gravity.attrs

In [9]:
bouguer = disturbance - hm.bouguer_correction(topography.topography)
bouguer.attrs = gravity.attrs

Merge all of the grids into one so we can save a single file.

In [10]:
grid = xr.merge([
    disturbance.to_dataset(name="gravity_disturbance"),
    bouguer.to_dataset(name="gravity_bouguer"),
    topography_smooth.topography.to_dataset(name="topography_smoothed"),
    topography.topography.to_dataset(name="topography"),
]).assign_coords(height=gravity.height)
grid

<xarray.Dataset>
Dimensions:              (latitude: 361, longitude: 721)
Coordinates:
  * longitude            (longitude) float64 -180.0 -179.5 ... 179.5 180.0
  * latitude             (latitude) float64 -90.0 -89.5 -89.0 ... 89.0 89.5 90.0
    height               (latitude, longitude) float64 2.716e+03 ... 15.47
Data variables:
    gravity_disturbance  (latitude, longitude) float64 -27.18 -27.18 ... 24.72
    gravity_bouguer      (latitude, longitude) float64 -331.2 -331.2 ... 312.6
    topography_smoothed  (latitude, longitude) float64 2.796e+03 ... -3.56e+03
    topography           (latitude, longitude) float64 2.716e+03 ... -4.213e+03

Export to netCDF using the netcdf4 engine which results in smaller files.

In [11]:
grid.to_netcdf(data_dir / "global-gravity.nc", format="NETCDF4", engine="netcdf4")